In [12]:
import mysql.connector
from enum import Enum

class DbConnection:
    connection = None

    def __init__(self):
        pass
    
    @classmethod
    def createDbConnection(self):
        if self.connection is not None:
            return self.connection
        
        self.connection = mysql.connector.connect(
            host='localhost',
            database='lms',
            user='root',
            password=''
        )
        
        if self.connection.is_connected():
            db_Info = self.connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
        
        return self.connection
    
    @classmethod
    def getCursor(self):
        return self.connection.cursor()

    @classmethod
    def commit(cls):
        cls.connection.commit()


class Users:
    
    def __init__(self, name, mobile):
        self.name = name
        self.mobile = mobile
    
    def create(self):
        statement = "INSERT INTO user (name, mobile) VALUES (%s, %s)"
        data = (self.name, self.mobile)
        cursor = DbConnection.getCursor()
        print(statement, data) 
        cursor.execute(statement, data)
        DbConnection.commit()

    @classmethod
    def getall(cls):
        statement = "SELECT * FROM user;"
        cursor = DbConnection.getCursor()
        cursor.execute(statement)
        result = cursor.fetchall()
        return result

    @classmethod
    def update_name(cls, name, id):
        statement = "UPDATE user set name = %s WHERE id = %s;"
        data = (name, id,)
        cursor = DbConnection.getCursor()
        cursor.execute(statement, data)
        DbConnection.commit()

    @classmethod
    def delete(self, id):
        statement = "DELETE FROM user WHERE id = %s;"
        data = (id,)
        cursor = DbConnection.getCursor()
        cursor.execute(statement, data)
        DbConnection.commit()
    
    def get_user_rent_fee(self, user_id):
        fee = 0
        statement = "SELECT DATEDIFF(rented_date, now()) FROM books where userId = %s and DATEDIFF(rented_date, now()) > 20;"
        data = (user_id,)
        cursor = DbConnection.getCursor()
        cursor.execute(statement, data)
        records = cursor.fetchall()
        for record in records:
            days = record[0]
            fee += 5*((days*(days+1)//2)-6)
        return fee
        
    
    @classmethod
    def get_input(cls):
        name = input("Enter username: ")
        mobile = input("Enter mobile number: ")
        return cls(name, mobile)


class Librarian:
    def __init__(self, name, mobile):
        self.name = name
        self.mobile = mobile
    
    def create(self):
        statement = "INSERT INTO librarian (name, mobile) VALUES (%s, %s)"
        data = (self.name, self.mobile)
        cursor = DbConnection.getCursor()
        print(statement, data) 
        cursor.execute(statement, data)
        DbConnection.commit()

    @classmethod
    def getall(cls):
        statement = "SELECT * FROM librarian;"
        cursor = DbConnection.getCursor()
        cursor.execute(statement)
        result = cursor.fetchall()
        return result

    @classmethod
    def update_name(cls, name, id):
        statement = "UPDATE librarian set name = %s WHERE id = %s;"
        data = (name, id,)
        cursor = DbConnection.getCursor()
        cursor.execute(statement, data)
        DbConnection.commit()

    @classmethod
    def delete(self, id):
        statement = "DELETE FROM librarian WHERE id = %s;"
        data = (id,)
        cursor = DbConnection.getCursor()
        cursor.execute(statement, data)
        DbConnection.commit()

    @classmethod
    def get_input(cls):
        name = input("Enter username: ")
        mobile = input("Enter mobile number: ")
        return cls(name, mobile)

class Books:
    
    def __init__(self, name, author, publication, id):
        self.id = None
        self.name = name
        self.author = author
        self.publication = publication


    def create(self):
        statement = "INSERT INTO books (name, author, publicatin) VALUES (%s, %s, %s)"
        data = (self.name, self.author, self.publication)
        cursor = DbConnection.getCursor()
        print(statement, data) 
        cursor.execute(statement, data)
        DbConnection.commit()

    @classmethod
    def getall(cls):
        statement = "SELECT * FROM books;"
        cursor = DbConnection.getCursor()
        cursor.execute(statement)
        result = cursor.fetchall()
        return result

    @classmethod
    def update_name(cls, name, id):
        statement = "UPDATE books set name = %s WHERE id = %s;"
        data = (name, id,)
        cursor = DbConnection.getCursor()
        cursor.execute(statement, data)
        DbConnection.commit()

    @classmethod
    def delete(self, id):
        statement = "DELETE FROM books WHERE id = %s;"
        data = (id,)
        cursor = DbConnection.getCursor()
        cursor.execute(statement, data)
        DbConnection.commit()
    
    def get_book_rent_fee(self, book_id):
        fee = 0
        statement = "SELECT DATEDIFF(rented_date, now()) FROM books where id = %s and DATEDIFF(rented_date, now()) > 20;"
        data = (book_id,)
        cursor = DbConnection.getCursor()
        cursor.execute(statement, data)
        records = cursor.fetchall()
        for record in records:
            days = record[0]
            fee += 5*((days*(days+1)//2)-6)
        return fee
    
    @classmethod
    def rent_to_user(self, userId, bookId):
        try:
            statement = "UPDATE books set user_id = %s, rented_date = now() WHERE id = %s and userId != %s;"
            data = (userId, bookId, userId)
            cursor = DbConnection.getCursor()
            cursor.execute(statement, data)
            DbConnection.commit()
        except:
            print("Book cannot be rented right now. Please try after couple of days.")
    
    def clear_rented_book(cls, bookId):
        statement = "UPDATE books set user_id = null, rented_date = null WHERE id = %s;"
        data = (bookId,)
        cursor = DbConnection.getCursor()
        cursor.execute(statement, data)
        DbConnection.commit()

    @classmethod
    def get_input(cls):
        name = input("Enter name: ")
        author = input("Enter author: ")
        publication = input("Enter publication: ")
        return cls(name, author, publication)



class Entity(Enum):
    USER = 'users'
    LIBRARIAN ='librarian'
    BOOK ='books'

print(Entity.USER.value)


DbConnection.createDbConnection()

def menu():
    ops = input("Select operation type(add, update, delete, read, rent):")
    if ops == 'add':
        entity = input('Select type of entity to add(users,librarian,books):')
        if entity == Entity.USER.value:
            user = Users.get_input()
            user.create()
        elif entity == Entity.LIBRARIAN.value:
            lib = Librarian.get_input()
            lib.create()
        elif entity == Entity.BOOK.value:
            books = Books.get_input()
            books.create()
    if ops == 'update':
        entity = input('Select type of entity to update(users,books):')
        if entity == Entity.USER.value:
            user_id = input('Enter user id: ')
            name = input('Enter user name')
            Users.update(name, user_id)
        elif entity == Entity.BOOK.value:
            book_id = input('Enter book id: ')
            name = input('Enter user name')
            Books.update(name, book_id)

    if ops == 'delete':
        entity = input('Select type of entity to add(users,librarian,books):')
        if entity == Entity.USER.value:
            user_id = input('Enter user id: ')
            Users.delete(user_id)
        elif entity == Entity.BOOK.value:
            book_id = input('Enter book id: ')
            Books.delete(book_id)

    if ops == 'read':
        print("Available books are:")
        for book in Books.getall():
            print(book[0], book[1], book[2], book[3])

    if ops == 'rent':
        user_id = input('Enter userId: ')
        book_id = input('Enter bookId: ')
        Books.rent_to_user(user_id, book_id)
    menu()

menu()

# a) add user, add librarian, add books.
# b) update user details, update books details
# c) delete user, delete books
# d) read the books details and list of books available
# e) perform rental operations


users
Connected to MySQL Server version  8.0.30


KeyboardInterrupt: Interrupted by user